In [25]:
pcks <- c('gsubfn', 'dplyr', 'tidyr', 'ggplot2', 'extrafont', 'stringr', 'ggpubr')
pcks <- sapply(pcks, require, character.only = 1)
if (any(! pcks)) install.packages(names(pcks)[! pcks])

p.vec1 = c(1, 1) / 2 # balanced
p.vec2 = c(2, 1) / 3 # unbalanced

p.list = list(p.vec1, p.vec2)

In [3]:
res <- NULL
for (i in 1:2) {
    for (q in c(5, 10)[i]) {
        for (dm in c(1e3, 2e3)[i]) {
            s.mat1 = diag(1, dm, dm)
            s.mat2 = 0.5^abs(outer(1:dm, 1:dm, "-"))
            s.list = list(s.mat1, s.mat2)
            for (s.mat in s.list[]) {
                cs.mat = t(chol(s.mat))
                for (p.vec in p.list[]) {
                  file.name <- fn$identity("p.`dm`.q.`q`/s.mat`s.mat[1, 3]`_prob`round(p.vec[1],digits=2)`_sim.valid.Rdata")
                  load(file.name)
                  message(file.name)
                  sim.result <- as.data.frame(sim.result)
                  sim.result$m <- c(rep('ERR', 3), rep('TPS', 3), rep('FPS', 3), rep('CMI', 3)) %>% 
                    factor(., levels = c('ERR', 'TPS', 'FPS', 'CMI'))
                  tmp <- cbind(s.mat=s.mat[1, 3], prob=round(p.vec[1],digits=2), sim.result)
                  res <- rbind(res, tmp)
                }
            }
        }
    }
}

s.mat0_prob0.5

s.mat0_prob0.67

s.mat0.25_prob0.5

s.mat0.25_prob0.67

s.mat0_prob0.5

s.mat0_prob0.67

s.mat0.25_prob0.5

s.mat0.25_prob0.67



In [4]:
colnames(res) <- c("s.mat", "prob", "p", "q", "n", 
                   "c.Bayes", "c.Lasso", "c.cLasso1", "c.cLasso2", "c.cLasso3", "c.cLasso4", "c.cLasso5", "c.Mcp1", "c.Mcp2", "c.Mcp3",
                   "Bayes", "LASSO", "MCL4", "MCL5", "MCL1", "MCL2", "MCL3", "MCP1", "MCP2", "MCP3",
                   "Measure")

In [71]:
plot.results <- function(s.mat0) {
    res_long <- res %>% mutate_at(vars(s.mat:MCP3), function(x) as.numeric(as.character(x))) %>% 
        filter(s.mat %in% s.mat0) %>% gather(methods, values, 
        c(LASSO, MCL1:MCL3, MCP1:MCP3), factor_key = TRUE) %>% 
        mutate(methods.g = case_when(str_detect(.$methods, "MCL") ~ 
            "MCL", str_detect(.$methods, "MCP") ~ "MCP", T ~ 
            "LASSSO"))
    n.vec <- res_long$n %>% unique
    q0 <- res_long$q[1]
    g.list <- list()
    options(repr.plot.width = 18, repr.plot.height = 13)
    for (m in res_long$Measure) {
        g.list[[m]] <- res_long %>% filter(Measure == m) %>% 
        ggplot(aes(x = n, y = values, group = methods))+
        facet_wrap( ~ p + prob, scales = "free", nrow = 1, labeller = label_both)+
        ylab(m) + geom_line(aes(linetype = methods, col = methods), lwd = 0.5)+
        scale_linetype_manual(values = c(1, 3, 3, 3, 5, 5, 5))+
        geom_point(aes(shape = methods, color = methods), size = 1.5)+
        scale_shape_manual(values = 1:7)+
        scale_color_manual(values = c("black", rep("blue", 3), rep("gold2", 3)))+
        scale_x_continuous(breaks = n.vec)+
        theme_bw(base_family = "Times New Roman", base_size = 10)
#         + theme(legend.position = ifelse(m == 'ERR', 'right', "none"))
    }
    g.list
}

In [72]:
g.list <- plot.results(0)

In [73]:
png('./s.0.png', res = 200, width = 22, height = 25, units = 'cm')
plot.results(.0) %>% ggarrange(plotlist = ., nrow = 4)
dev.off()
png('./s.0.25.png', res = 200, width = 22, height = 25, units = 'cm')
plot.results(.25) %>% ggarrange(plotlist = ., nrow = 4)
dev.off()

pdf 
  2

pdf 
  2